In [1]:
import os

import keras
from keras import layers
from keras import ops
import numpy as np

In [2]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def compute_loss(
        self, x=None, y=None, y_pred=None, sample_weight=None, allow_empty=False
    ):
        teacher_pred = self.teacher(x, training=False)
        student_loss = self.student_loss_fn(y, y_pred)

        distillation_loss = self.distillation_loss_fn(
            ops.softmax(teacher_pred / self.temperature, axis=1),
            ops.softmax(y_pred / self.temperature, axis=1),
        ) * (self.temperature**2)

        loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss
        return loss

    def call(self, x):
        return self.student(x)

In [3]:
teacher = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(512, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="teacher",
)


student = keras.Sequential(
    [
        keras.Input(shape=(28, 28, 1)),
        layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(negative_slope=0.2),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="student",
)

student_scratch = keras.models.clone_model(student)

In [4]:
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))

In [5]:
teacher.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

teacher.fit(x_train, y_train, epochs=5)
teacher.evaluate(x_test, y_test)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 307s 159ms/step - loss: 0.2535 - sparse_categorical_accuracy: 0.9235
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 246s 131ms/step - loss: 0.0911 - sparse_categorical_accuracy: 0.9730
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 240s 128ms/step - loss: 0.0751 - sparse_categorical_accuracy: 0.9770
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 236s 126ms/step - loss: 0.0635 - sparse_categorical_accuracy: 0.9808
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 291s 155ms/step - loss: 0.0616 - sparse_categorical_accuracy: 0.9823
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - loss: 0.1061 - sparse_categorical_accuracy: 0.9748


[0.09291920065879822, 0.9779000282287598]

In [6]:
distiller = Distiller(student=student, teacher=teacher)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

distiller.fit(x_train, y_train, epochs=3)

distiller.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 347s 183ms/step - loss: 1.6315 - sparse_categorical_accuracy: 0.7283
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 215s 115ms/step - loss: 0.0311 - sparse_categorical_accuracy: 0.9491
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 212s 113ms/step - loss: 0.0214 - sparse_categorical_accuracy: 0.9619
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - loss: 0.0191 - sparse_categorical_accuracy: 0.9626


[0.01732574589550495, 0.9692000150680542]

In [7]:
student_scratch.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

student_scratch.fit(x_train, y_train, epochs=3)
student_scratch.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 0.4423 - sparse_categorical_accuracy: 0.8656
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.0998 - sparse_categorical_accuracy: 0.9696
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0794 - sparse_categorical_accuracy: 0.9765
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0825 - sparse_categorical_accuracy: 0.9728


[0.06602774560451508, 0.9789000153541565]

In [ ]:
#  conclusion 

# the teacher to have accuracy around 97.6%, the student trained from scratch should be around 97.6%,
#  and the distilled student should be around 96.9%. 
